## Introduction

This notebook contains workflows to derive RESP-dPol from baseline QM ESPs.

See [documentation](https://factorpol.readthedocs.io/en/latest/api.html#factorpol.charge_training.ChargeTrainer.derive_resp_dpol) of RESP-dPol fitting for details.

In [1]:
import os
import numpy as np
import pandas as pd
from openff.toolkit import ForceField
from pkg_resources import resource_filename

from factorpol.charge_training import ChargeTrainer

from factorpol.utilities import (
    Polarizability,
    StorageHandler,
    retrieve_records,
    calc_rrms,
)

cwd = os.getcwd()
off_forcefield = ForceField(
    resource_filename("factorpol", os.path.join("data", "off_examples.offxml"))
)

/home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/wwilla/mambaforge/envs/factorpol/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


## Prepare data

In [2]:
dataset = ["CO"]

Create a `sqlalchemy` session

In [3]:
st = StorageHandler()
ses = st.session("factorpol_examples")

In [4]:
esp_records = retrieve_records(my_session=ses, dataset=dataset)

In [5]:
print(f"Number of ESP records:\t {len(esp_records)}")

Number of ESP records:	 1


Load polarizabilities we derived in the previous example `01-derive-polarizability`

In [6]:
polarizability = Polarizability(data_source="ret_alphas.csv")

In [7]:
polarizability.data

,Polarizability (angstrom**3)
Type,
[#1:1],0.377140
[#6:1],1.440485
[#7:1],0.000000
[#8:1],0.000000


### RESP-dPol

In [8]:
methanol = ChargeTrainer(
    record=esp_records["CO"],
    polarizability=polarizability,
    off_forcefield=off_forcefield,
    coulomb14scale=0.5,
)

/home/wwilla/data_main/infrastructure/factorpol/factorpol/charge_training.py:82: RuntimeWarning: divide by zero encountered in power
  self._r_jk == 0.0, self._r_jk, np.power(self._r_jk, -3)


In [9]:
print("Derive RESP-dPol partial charges for methanol")
methanol.resp_dpol

Derive RESP-dPol partial charges for methanol


Magnitude,[0.08855988896282083 -0.607709416056741 0.04699130610636622 0.04699130610636623 0.046991306106366225 0.3793970905121749]
Units,elementary_charge


### Calculate Quality-of-fit RRMS

In [10]:
rrms = calc_rrms(calc=methanol.mm_dpol_esps.magnitude, ref=methanol.esp_values)
print(f"Quality-of-fit RRMS = {rrms:.3f}")

Quality-of-fit RRMS = 0.011
